# Lab 01 · FastAPI, Vite, and Git

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A FastAPI backend is scaffolded with health and echo routes.
- A Vite React frontend is outlined with modern tooling defaults.
- A Git repository is initialized with environment templates.

## What will be learned
- The structure of a basic FastAPI service is reviewed.
- Development-time CORS settings are configured.
- Vite-powered React scaffolding steps are rehearsed.
- Git initialization practices are reinforced.


### FastAPI framework primer

**FastAPI in a nutshell.** [FastAPI](https://fastapi.tiangolo.com/) is an asynchronous Python web framework that automatically maps HTTP paths to Python callables called *path operations*. Typed request and response models built with [Pydantic](https://docs.pydantic.dev/) provide validation, while async support keeps expensive AI inference calls from blocking the event loop.

#### Core building blocks
1. **Application entry point** — Create a `FastAPI()` instance in `app/main.py` and register startup/shutdown handlers to open or close model clients. [Docs: First Steps](https://fastapi.tiangolo.com/tutorial/first-steps/)
2. **Routers and path operations** — Organize endpoints in `routers/` modules with `APIRouter` to keep chat, embeddings, and admin surfaces separate. Mount each router in `main.py` to expose `/api/chat`, `/api/embeddings`, etc. [Docs: APIRouter](https://fastapi.tiangolo.com/tutorial/bigger-applications/)
3. **Pydantic models** — Declare request/response schemas in `schemas.py` so prompts, system messages, and model options are validated before invoking inference. [Docs: Request Body](https://fastapi.tiangolo.com/tutorial/body/)
4. **Dependency injection** — Use `Depends` to pass shared services (OpenAI client, vector store, cache) into route functions without manually instantiating them. This keeps AI integrations testable and configurable. [Docs: Dependencies](https://fastapi.tiangolo.com/tutorial/dependencies/)
5. **Background tasks** — Offload long-running work such as audit logging, metrics collection, or follow-up summarization via `BackgroundTasks`. Students can immediately see how to chain AI workflows beyond the initial response. [Docs: Background Tasks](https://fastapi.tiangolo.com/tutorial/background-tasks/)
6. **Interactive OpenAPI docs** — FastAPI generates `/docs` and `/redoc` from your type hints, letting you verify model contracts, stream responses, and share live documentation with teammates. [Docs: Interactive API docs](https://fastapi.tiangolo.com/features/#interactive-api-docs)

#### Tooling and project layout for AI routes
- **Local server:** Run `uvicorn app.main:app --reload` for fast feedback while editing routers or schemas. Enable hot reload so AI inference tweaks are visible immediately. [Docs: Uvicorn](https://fastapi.tiangolo.com/deployment/uvicorn/)
- **Environment management:** Load `.env` files with `python-dotenv` or FastAPI settings management to inject API keys, base URLs, and model IDs without hardcoding secrets.
- **Folder structure:** Start with `app/main.py`, `app/routers/`, `app/schemas.py`, and `app/services/ai.py`. Keep retry utilities, tracing helpers, and vector database adapters in `services/` so routes remain thin controllers.
- **Testing hooks (Optional):** Leverage FastAPI's `TestClient` to write unit tests that mock AI providers and confirm each router handles success, errors, and streaming payloads.

### Vite + React framework primer

#### Why Vite for React AI UIs
The frontend labs rely on [Vite](https://vitejs.dev/guide/) to scaffold a React UI with lightning-fast hot module replacement, native ES module bundling, and first-class TypeScript. CLI commands such as `npm create vite@latest`, `npm install`, `npm run dev`, and `npm run build` (or their `pnpm`/`yarn` equivalents) keep iteration tight while preparing optimized builds for deployment.

#### Core tooling and project conventions
- **Dev server workflow:** `npm run dev` launches a Vite server that can [proxy](https://vitejs.dev/config/server-options.html#server-proxy) `/api` calls to FastAPI, eliminating CORS friction while you develop AI features.
- **Environment variables:** Prefix keys with `VITE_` (e.g., `VITE_API_BASE_URL`, `VITE_OPENAI_MODEL`) so they are exposed via `import.meta.env`. Use separate `.env.local` files for experimental model toggles.
- **Module organization:** Mirror backend routers with feature folders like `src/features/chat/`, `src/features/embeddings/`, and keep shared helpers under `src/lib/` (`apiClient.js`, `retry.js`, `streamParser.js`).
- **React hooks and state:** Follow the [React documentation](https://react.dev/learn) to combine `useState`, `useEffect`, and custom hooks that orchestrate fetch lifecycles. Patterns such as `useReducer` for complex prompt builders or `useRef` for streaming buffers keep AI UIs maintainable.
- **UI composition:** Component libraries (Mantine, Chakra) or design systems can slot into `src/components/` for reusable chat bubbles, token counters, and loading spinners.

#### Mini walkthrough: bootstrapping the AI stack
1. **Scaffold FastAPI** with `python -m venv .venv && pip install fastapi uvicorn` and create `app/main.py` that instantiates the app, registers routers, and mounts static files if needed.
2. **Add routers and schemas** under `app/routers/inference.py` with `Pydantic` models describing your prompt payloads and inference responses.
3. **Wire dependencies** like model clients via `Depends`, storing credentials in `.env` and loading them with a settings class.
4. **Run `uvicorn app.main:app --reload`** to expose `/docs`, test path operations, and confirm background tasks fire during inference.
5. **Create the Vite React frontend** with `npm create vite@latest frontend -- --template react` and install UI dependencies that render form controls and streaming areas.
6. **Configure Vite proxy and envs** in `vite.config.js` and `.env.local` so `fetch` helpers in `src/lib/apiClient.js` point to FastAPI during development.
7. **Implement React hooks and components** (e.g., `useInference` in `src/features/chat/useInference.js`) that call the backend, manage retries, and render streaming updates.
8. **Iterate locally** by running `uvicorn` and `npm run dev` together, validating requests through the autogenerated FastAPI docs and the Vite dev tools before containerization or deployment.



## Prerequisites & install
The container workflow depends on these locally installed tools:

```bash
docker --version
docker compose version
git --version
```

After cloning the repository, build and start the services with Docker Compose:

```bash
cd ai-web
docker compose build
docker compose up
# Backend → http://localhost:8000
# Frontend → http://localhost:5173

# Shut the stack down when finished exploring:
docker compose down
```

## Step-by-step tasks
Each step configures local source files that the Docker stack will mount so secrets remain outside the built images.

### Step 1: Backend folder layout and Dockerfile
A backend folder is created with starter FastAPI files and a Dockerfile that installs dependencies inside the image.

### Step 1: Explore the FastAPI service + router scaffold

The repository already contains the backend scaffold that students will mirror
in class. Walk through the existing files inside `ai-web/backend/app/` so
instructors understand how the service and router layers stay separated:

1. **`services/echo.py`** – Demonstrates keeping retry simulation logic out of
   the HTTP layer. Highlight the `_FLAKY_ATTEMPTS` dictionary and
   `get_flaky_echo_payload` helper so learners see where transient errors are
   raised.

   ```python
   from app.services.echo import EchoServiceError, get_flaky_echo_payload
   ```

   Emphasize that routers import helpers like this instead of owning the state
   themselves.
2. **`routers/echo.py`** – Contains the thin `/echo` and `/flaky-echo` handlers.
   Point out the `EchoIn` Pydantic model and the fact that `flaky_echo` catches
   `EchoServiceError` and converts it into an HTTP 503. This mirrors how future
   AI features will raise custom exceptions from their services.
3. **`main.py`** – Keeps application assembly minimal: load environment
   variables, add CORS middleware, and register routers.

   ```python
   load_dotenv()
   app = FastAPI()
   app.add_middleware(CORSMiddleware, allow_origins=["http://localhost:5173"], allow_methods=["*"], allow_headers=["*"])
   app.include_router(echo_router)
   app.include_router(gemini_router)
   ```

4. **Environment files** – Copy `backend/.env.example` to `backend/.env` so the
   compose stack can mount real secrets. Document new keys in the `.env.example`
   template first; the Docker volume defined later binds the populated `.env`
   into the container at runtime.
5. **`requirements.txt`** – Already includes `google-generativeai`, `python-dotenv`,
   and other dependencies referenced in later labs. Reinforce that these files
   stay in sync with the walkthrough so students are not asked to install packages
   manually during lectures.

Close the backend tour by opening `docs/feature-workflow.md` with the class; it
summarizes the same service → router → feature flow they are about to practice.


### Step 2: Understand the React feature-folder pattern

Rather than editing a monolithic `App.jsx`, the frontend ships with feature
folders under `frontend/src/features/`. Each folder contains a hook for data
fetching/state and a component for rendering. Review the echo feature to show
how the pattern works:

- **Hook (`features/echo/hooks/useEchoForm.js`)** – Calls the backend via
  `withRetry`, manages loading/error state, and exposes a simple API to the
  component. Draw attention to how the hook builds the `/flaky-echo` URL with a
  `failures` query parameter so instructors know where to tweak simulations.
- **Component (`features/echo/components/EchoForm.jsx`)** – Receives props from
  the hook and stays presentational. Discuss the `prop-types` definitions and
  the way the component renders instructional copy alongside the form so
  learners see a well-documented example.
- **App shell (`src/App.jsx`)** – Imports the hook/component pairs and simply
  composes them inside the layout.

Encourage instructors to reuse this structure when introducing new labs: start
with a hook next to the feature, keep `App.jsx` tiny, and colocate any helper
modules inside `src/lib/`.


### Step 3: Trace the frontend → backend integration points

Help students connect the dots between React hooks and FastAPI endpoints by
examining the shared helpers:

1. **`src/lib/api.js`** – Centralizes the `fetch` wrapper and pulls the API base
   URL from `VITE_API_BASE`. Highlight how the error-handling branch reads JSON
   `detail` fields so retry failures surface as friendly messages.
2. **`src/lib/retry.js`** – The `withRetry` helper now guards the final sleep by
   checking `if (attempt < attempts)` before awaiting `setTimeout`. Show the
   class how this prevents an unnecessary delay after the last try.
3. **`features/gemini/hooks/useLessonOutlineForm.js`** – Mirrors the echo hook
   but posts to `/ai/lesson-outline`. Use this as a preview of Lab 03 so the
   transition from echo to Gemini feels natural.

After walking through the code, open the running dev servers and demonstrate how
submitting the echo form triggers the `/flaky-echo` endpoint. Then flip to the
Gemini form and highlight the shared loading/error patterns.


### Step 4: Review Docker Compose orchestration

A top-level `docker-compose.yml` starts the backend and frontend together. The
file in the repository already matches what students need; use it as a teaching
artifact instead of rewriting it live.


### Step 5: Map Compose services to the codebase

Walk line-by-line through `ai-web/docker-compose.yml` so instructors know how
local edits reach the containers.

```yaml
version: "3.9"

services:
  backend:
    build: ./backend
    ports:
      - "8000:8000"
    volumes:
      - ./backend/app:/app/app
      - ./backend/.env:/app/.env:ro

  frontend:
    build: ./frontend
    ports:
      - "5173:5173"
    environment:
      - VITE_API_BASE=http://localhost:8000
    volumes:
      - ./frontend/src:/app/src
      - ./frontend/vite.config.js:/app/vite.config.js
      - ./frontend/index.html:/app/index.html
      - ./frontend/package.json:/app/package.json
```

Call out that mounting `backend/.env` lets FastAPI read secrets without baking
them into the image, and that the frontend mounts `src/` so hot module reloads
work during demos.


### Step 4: Git initialization
Git is initialized locally so changes can be tracked.

### FastAPI ↔ Vite AI workflow checklist

Use this quick-reference list alongside `docs/feature-workflow.md` whenever you
introduce a new full-stack capability:

1. Start with a service module under `app/services/` that encapsulates business
   logic and external SDK calls. Document custom exceptions so routers can
   translate them into HTTP responses.
2. Add a router in `app/routers/` with request/response models, then register it
   in `app/main.py` via `app.include_router(...)`.
3. Create a hook + component pair under `frontend/src/features/` and wire it
   into `src/App.jsx`. Store shared utilities in `src/lib/`.
4. Capture any new configuration in `backend/.env.example` and
   `frontend/.env.example`, then update real `.env` files before running Docker
   Compose.
5. Test the API with `curl` and the UI with `npm run dev` to confirm both ends
   handle success, validation errors, and simulated outages (such as hitting
   `/flaky-echo?failures=3`).


```bash
cd ai-web
git init
git add .
git commit -m "Lab 1 scaffold"
```

## Validation / acceptance checks
```bash
# locally
docker compose up -d
curl http://localhost:8000/health
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"hello"}'
docker compose down
```
- HTTP 200 responses include status "ok" and the echoed payload while the stack runs in Docker.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- A README entry is expanded to document backend and frontend start commands.
- A GitHub repository is connected for remote backups.